In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle

In [136]:
curr_dir = !pwd
domain_name = "Homeobox"
input_path = curr_dir[0]+"/../5.HMM_alter_align/domains_states_dicts/pfam_v30/"+domain_name+"/"
dirfiles = !ls -t $input_path
filename = dirfiles[0]

with open(input_path+filename, 'rb') as handle:
    states_dict = pickle.load(handle)
    
#Getting the number of states for this domain
states_num = len(states_dict.keys())

In [137]:
MAF_thresholds = [0.05, 0.005, 0.0005, 0.00005, 0.000005]

rare_pol_dict = defaultdict(list)
for maft in MAF_thresholds:
    for state in states_dict.keys():
        pol_sites_num = 0
        pol_rare_num = 0
        for d in states_dict[state]:
            if (d["af_adj"] > 0):
                pol_sites_num += 1
                if (d["af_adj"] <= maft):
                    pol_rare_num += 1
        if (pol_sites_num == 0):
            rare_pol_frac = 0
        else:
            rare_pol_frac = pol_rare_num/float(pol_sites_num)
        rare_pol_dict[state].append(rare_pol_frac)

#Adding number of SNPs in each state
for state in states_dict.keys():
    pol_sites_num = 0
    for d in states_dict[state]:
        if (d["af_adj"] > 0):
            pol_sites_num += 1
    rare_pol_dict[state].append(pol_sites_num)

In [138]:
#Exporting to data-frames tables
rare_poly_df = pd.DataFrame.from_dict(rare_pol_dict,orient='index')
rare_poly_df.columns = ["0.05", "0.005", "0.0005", "0.00005","0.000005", "number"]
rare_poly_df["state"] = rare_pol_dict.keys()

rare_poly_df.to_csv(curr_dir[0]+"/data-frames/rare_poly/"+domain_name+"_poly.csv", sep='\t')

In [118]:
total_snp = 0
rare_snp = 0
sum_snp = 0
for state in states_dict.keys():
    for d in states_dict[state]:
        if (d["af_adj"] > 0):
            total_snp += 1
            sum_snp += d["af_adj"]
            if (d["af_adj"] < 0.00005):
                rare_snp += 1
                
rare_pol_frac = rare_snp/float(total_snp)
af_avg = sum_snp/float(total_snp)

In [99]:
rare_pol_frac

0.8229537366548043

In [100]:
rare_snp

1850

In [85]:
total_snp

2248

In [119]:
sum_snp

0.002131481

In [120]:
af_avg

2.804580263157895e-05